In [1]:
from typing import List

import pandas as pd
from pandas import DataFrame

import geocoder

# DataFrame creation

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
dfs: List[DataFrame] = pd.read_html(url)

I read first the data from the given url, web scrapping is made by pandas :smile:

In [3]:
df, *_ = pd.read_html(url, na_values="Not assigned")

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [4]:
df.columns = ['PostalCode', 'Borough', 'Neighborhood']

I process the cells that have an assigned borough and ignore cells with a borough that is Not assigned.

In [5]:
df = df[~df.Borough.isna()]

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

`.groupby()` use by creating a function doing that ope

In [6]:
def f(group):
    # If a cell has a borough but a Not assigned neighborhood,
    # then the neighborhood will be the same as the borough.
    group['Neighborhood'] = group['Neighborhood'].fillna(group.Borough)
    return group

Reset the index and drop the former index values

In [7]:
df.groupby('PostalCode').apply(f).reset_index(drop=True).head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Number of rows of the dataframe.

In [8]:
df.groupby('PostalCode').apply(f).shape[0]

103

In [9]:
gdf = df.groupby('PostalCode').apply(f).reset_index(drop=True)
gdf.head(2)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village


# Latitude and longitude

Using the PostalCode and Borough data from the dataframe to build a query string for google geocoding API

In [10]:
search_query = ", ".join(gdf.iloc[0][["PostalCode", "Borough"]])
search_query

'M3A, North York'

Unfortunalty, no [API key](https://geocoder.readthedocs.io/providers/Google.html#environment-variables) available

In [11]:
geocoder.google(search_query)

<[REQUEST_DENIED] Google - Geocode [empty]>

I go for the csv data and rename one column

In [12]:
gd: DataFrame = pd \
    .read_csv("https://cocl.us/Geospatial_data") \
    .rename(mapper={'Postal Code': 'PostalCode'}, axis=1)
gd.head(2)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497


Join the two DataFrame

In [13]:
dfc = gdf.set_index("PostalCode").join(gd.set_index("PostalCode")).reset_index()
dfc.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
